# Day 6 Session 1 – Integrating Vision with Simulation

Robots often rely on **computer vision** to understand the world. In earlier sessions (Day 3), we worked with OpenCV to detect colored objects in images. Now, we’ll discuss how to connect that with our simulation world – essentially, how to map a position in an image (pixel coordinates) to a position in the simulation (world coordinates).

This is a challenging topic because it involves **camera calibration** and understanding geometry:
- A camera sees an object at some pixel (x, y) in an image.
- We want to know where that object is in the robot’s coordinate system (e.g., on the floor or table in front of the robot).


## The Challenge: Pixels to World Coordinates
Imagine you have a camera mounted above a robot’s workspace (like looking down at a table). You use OpenCV to detect a red ball in the camera image at pixel position (px, py). How do we tell the robot arm where that ball is on the table in X, Y coordinates (and maybe Z if height)?

In an ideal simple case (like a camera looking straight down, and we know the area it covers), we can use a **linear mapping** from pixel coordinates to world coordinates:

- Suppose the camera image covers a region of the table from X_min to X_max and Y_min to Y_max.
- px = 0 might correspond to X_min, px = image_width corresponds to X_max.
- Similarly for py to Y (but be careful: in images, y increases downward, whereas in a typical world coordinate, Y might increase upward or some axis – we have to account for that flip).

<img src="https://i.postimg.cc/GmPJ0JTF/Screenshot-2025-06-12-at-12-59-09-AM.png" alt="Sample Image" width="250"/>



This linear mapping essentially assumes a flat plane and an orthographic projection (no distortion or perspective – which is an approximation, but okay for a fixed downward camera after calibration).

**Calibration** is the process of finding the exact mapping. Often you might place known markers (like a checkerboard or fiducials) in the scene, take images, and compute the relationship between image pixels and real-world coordinates. That can yield a transformation (like a homography for a flat surface) that maps image coordinates to world coordinates.

For simplicity, let’s assume we have done a calibration and know:
- The camera image of width `W` and height `H` corresponds to an area from X = `x_min` to x_max, Y = `y_min` to `y_max` in world coordinates (on the plane of the table, Z fixed).

Then the mapping for any detected pixel (px, py) is:
```
norm_x = px / W
norm_y = py / H
world_X = x_min + norm_x * (x_max - x_min)
world_Y = y_max + -norm_y * (y_max - y_min)
```
Notice for Y we do `y_max + -norm_y * (y_max - y_min)`. This effectively maps py=0 (top of image) to world_Y = y_max, and py=H to world_Y = y_min. We inverted the Y because image Y is top->down, whereas we define world Y increasing in the opposite direction (assuming y_max is top edge of the area).

The result (world_X, world_Y) tells us where on the table (in robot coordinates) the object is. The robot can then use that as a target for reaching, etc. (Usually, we’d also need the world Z, which might be known if on table surface, e.g., Z=table height.)

## Understanding Through an Example
Consider a 640x480 camera image. Suppose through calibration we determine that:
- The image’s left corresponds to X = -0.5m, right corresponds to X = +0.5m.
- The image’s top corresponds to Y = +0.5m, bottom to Y = -0.5m (assuming origin at center of table).

Thus:
- x_min = -0.5, x_max = 0.5
- y_min = -0.5, y_max = 0.5
- W = 640, H = 480.

If OpenCV detects an object at pixel (px=320, py=240) (the center of the image), the mapping gives:
```
norm_x = 320/640 = 0.5
norm_y = 240/480 = 0.5
world_X = -0.5 + 0.5*(0.5 - (-0.5)) = -0.5 + 0.5*1.0 = 0.0  (the midpoint in X)
world_Y = 0.5 + -0.5*(0.5 - (-0.5)) = 0.5 + -0.5*1.0 = 0.0  (midpoint in Y)
```
So (0,0) in world – which makes sense: the object is in the image center corresponds to table center.

If an object is at the top-left of the image (px=0, py=0):
```
norm_x=0, norm_y=0
world_X = -0.5 + 0*(1.0) = -0.5
world_Y = 0.5 + -0*1.0 = 0.5
```
So that maps to (-0.5, 0.5) in world – the top-left of the table area.

This linear mapping is essentially a scale and shift, which is one form of calibration if the camera is perfectly top-down and undistorted.

In reality, if the camera is angled or lenses cause distortion, you’d need a more complex model (like using camera intrinsics and extrinsics to do a full projection mapping). But those details involve pinhole camera models and are beyond our current scope. Our linear mapping is a simplification that works in a controlled setup.


## Sources of Error and Why Calibration Matters

If our mapping is off – say the camera was not exactly centered or the scaling is wrong – the world coordinate we compute could be several centimeters off from the real object position. This could cause a robot arm to miss when trying to grab something. Factors like:
- Camera tilt (if not exactly top-down, the mapping gets skewed).
- Lens distortion (straight lines in reality appear curved in image).
- Incorrect identification of the region (if the camera view covers a larger or smaller area than assumed).

These are handled by calibration:
- We would use known reference points to solve for a transformation that best maps pixel to world coordinates (this could be a homography in a planar case).
- We might also account for lens distortion by applying an undistortion step to the image before mapping.

For now, keep in mind: **garbage in**, **garbage out**. If the vision system’s coordinate mapping isn’t calibrated right, the robot will go to the wrong place. This is why robotics engineers spend time calibrating cameras to robots (through techniques like hand-eye calibration, etc.).

In the next hands-on session, we will assume a simple calibrated scenario and actually map a detected pixel to a world coordinate in our PyBullet sim.